In [1]:
import tushare as ts
ts.set_token("4dfe93632a16f49cae109f45465cc2aa13e6151e3a879cfa23d71d72")
pro = ts.pro_api()


df = pro.daily(ts_code='000001.SZ', start_date='20170401', end_date='20180728')
print(df)

       ts_code trade_date  open  high   low  close  pre_close  change  \
0    000001.SZ   20180727  9.30  9.36  9.18   9.25       9.25    0.00   
1    000001.SZ   20180726  9.40  9.48  9.21   9.25       9.38   -0.13   
2    000001.SZ   20180725  9.44  9.47  9.33   9.38       9.40   -0.02   
3    000001.SZ   20180724  9.39  9.59  9.33   9.40       9.42   -0.02   
4    000001.SZ   20180723  9.04  9.45  9.00   9.42       9.11    0.31   
..         ...        ...   ...   ...   ...    ...        ...     ...   
319  000001.SZ   20170411  9.17  9.19  9.09   9.15       9.18   -0.03   
320  000001.SZ   20170410  9.20  9.21  9.17   9.18       9.20   -0.02   
321  000001.SZ   20170407  9.19  9.22  9.17   9.20       9.20    0.00   
322  000001.SZ   20170406  9.20  9.22  9.17   9.20       9.21   -0.01   
323  000001.SZ   20170405  9.16  9.22  9.15   9.21       9.17    0.04   

     pct_chg         vol       amount  
0     0.0000   515126.03   477527.419  
1    -1.3859   833358.15   778429.753  
2  

In [4]:
# ARIMA模型
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

def model(a, b, c, m, df):
    # a, b, c是ARIMA模型的参数,a是自回归项,b是差分项,c是移动平均项
    # m是训练集的占比
    # df是数据集
    # 返回模型的训练结果和预测结果
    df = df['close'].values
    train_size = int(len(df) * m)
    train, test = df[0:train_size], df[train_size:]
    history = [x for x in train]
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=(a, b, c))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
    error = mean_squared_error(test, predictions)
    print('Test MSE: %.3f' % error)
    if error < 1:
        print('模型表现较好')
    else:
        print('模型表现较差')
    # plt.plot(test)
    # plt.plot(predictions, color='red')
    # plt.show()
    #预测接下来20天的数据
    forecast = model_fit.forecast(20)
    # plt.plot(forecast)
    # plt.show()
    print("预测接下来20天的数据：",forecast)
    #若下个20天的平均值大于今天的收盘价，则买入，否则卖出
    # if np.mean(forecast[0])>df[-1]:
    #     print(1)
    # else:
    #     print(0)
    return int(np.mean(forecast[0])>df[-1]),error,forecast

In [5]:
model(2, 1, 1, 0.8, df)
#2为自回归项，1为差分项，1为移动平均项，0.8为训练集占比，df为数据集
#输出值为均方误差,模型表现，预测值，是否买入


Test MSE: 0.005
模型表现较好
预测接下来20天的数据： [9.2018552  9.20204847 9.20222776 9.20226302 9.20228207 9.20228708
 9.20228923 9.20228989 9.20229014 9.20229022 9.20229026 9.20229027
 9.20229027 9.20229027 9.20229027 9.20229027 9.20229027 9.20229027
 9.20229027 9.20229027]


(0,
 0.004807909315323026,
 array([9.2018552 , 9.20204847, 9.20222776, 9.20226302, 9.20228207,
        9.20228708, 9.20228923, 9.20228989, 9.20229014, 9.20229022,
        9.20229026, 9.20229027, 9.20229027, 9.20229027, 9.20229027,
        9.20229027, 9.20229027, 9.20229027, 9.20229027, 9.20229027]))

In [7]:
# 朴素法 (Modified with Technical Analysis)

def model(df, m):
    df = df[['close', 'vol']]
    last_day_data = df.iloc[-1]
    yesterday_data = df.iloc[-2]
    
    # 计算支撑线和阻力线，取训练集百分比的数据
    train_size = int(len(df) * m)
    data1 = df[-train_size:]
    support_line = data1['close'].min()
    resistance_line = data1['close'].max()
    
    # 比较最后一天的收盘价和支撑线、阻力线的关系,且比较成交量
    if last_day_data['close'] < support_line and last_day_data['vol'] > yesterday_data['vol']:
        print(1)
    elif last_day_data['close'] > resistance_line and last_day_data['vol'] > yesterday_data['vol']:
        print(0)
    else:
        print(0)

    #测试集进行测试，计算均方误差
    train, test = df[0:train_size], df[train_size:]
    history = [x for x in train['close']]
    predictions = list()
    for t in range(len(test)):
        yhat = history[-1]
        predictions.append(yhat)
        obs = test['close'].values[t]
        history.append(obs)
    error = mean_squared_error(test['close'], predictions)
    if error < 1:
        print(1)
    else:
        print(0)
    #预测接下来20天的数据
    forecast = np.mean(test['close'].values[-20:])
    print("预测接下来20天的数据：",forecast)
    # plt.plot(forecast)
    # plt.show()
    #若下个20天的平均值大于今天的收盘价，则买入，否则卖出
    if forecast>df['close'].values[-1]:
        print(1)
    else:
        print(0)
    return error,forecast,int(forecast>df['close'].values[-1])

model(df, 0.8)
#df为数据集，0.8为训练集占比
#输出值为均方误差,模型表现，预测值，是否买入





0
1
预测接下来20天的数据： 9.046999999999999
0


(0.005267692307692316, 9.046999999999999, 0)